<a href="https://colab.research.google.com/github/Arminhosseini/CNN_mnist/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import tensorflow.keras as keras
import numpy as np

In [ ]:
num_classes = 10 # total classe (0-9 digits)

learning_rate = 0.001
training_steps = 200
batch_size = 128
display_steps = 10


conv1_filters = 32 # number of filters for the first conv layer
conv2_filters = 64 # number of filters for the second conv layer
fcl_units = 1024 # number of neurons for the fully connected layer

In [ ]:
from keras.datasets import mnist
(x_train, y_train) , (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset shapes: ((None, 28, 28), (None,)), types: (tf.float32, tf.uint8)>


In [ ]:
class ConvNet(Model):
  def __init__(self):
    super(ConvNet, self).__init__()

    self.conv1 = layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu)

    self.maxpool1 = layers.MaxPool2D(2, strides=2)

    self.conv2 = layers.Conv2D(64, kernel_size=3, activation=tf.nn.relu)

    self.maxpool2 = layers.MaxPool2D(2, strides=2)

    self.flatten = layers.Flatten()

    self.fc1 = layers.Dense(1024)

    self.dropout = layers.Dropout(rate = 0.2)

    self.fc2 = layers.Dense(2048)

    self.dropout2 = layers.Dropout(rate = 0.5)

    self.out = layers.Dense(num_classes)


  def call (self, x, is_training = False):
    x = tf.reshape(x, [-1,28,28,1])
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x, training = is_training)
    x = self.fc2(x)
    x = self.dropout2(x, training = is_training)
    x = self.out(x)


    if not is_training:
      x = tf.nn.softmax(x)

    
    return x



conv_net = ConvNet()  

In [ ]:
def cross_entropy_loss(x,y):
  y = tf.cast(y, tf.int64)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
  return tf.reduce_mean(loss)


def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)


optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

## optimization process

In [ ]:
@tf.function
def run_optimization(x, y):
  with tf.GradientTape() as tape:
    pred = conv_net(x, is_training = True)
    loss = cross_entropy_loss(pred, y)

  trainable_variables = conv_net.trainable_variables
  gradients = tape.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients,trainable_variables))

## Run training for the given number of steps

In [ ]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  run_optimization(batch_x, batch_y)

  if step % display_steps == 0:
    pred = conv_net(batch_x)
    loss = cross_entropy_loss(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print('step: %i, loss: %f, accuracy: %f'%(step, loss, acc))

step: 10, loss: 1.665745, accuracy: 0.851562
step: 20, loss: 1.600651, accuracy: 0.914062
step: 30, loss: 1.573416, accuracy: 0.921875
step: 40, loss: 1.529433, accuracy: 0.968750
step: 50, loss: 1.528706, accuracy: 0.945312
step: 60, loss: 1.530127, accuracy: 0.953125
step: 70, loss: 1.496995, accuracy: 0.984375
step: 80, loss: 1.513624, accuracy: 0.984375
step: 90, loss: 1.518759, accuracy: 0.968750
step: 100, loss: 1.490051, accuracy: 0.984375
step: 110, loss: 1.499300, accuracy: 0.984375
step: 120, loss: 1.506107, accuracy: 0.976562
step: 130, loss: 1.514315, accuracy: 0.960938
step: 140, loss: 1.488542, accuracy: 0.984375
step: 150, loss: 1.473086, accuracy: 1.000000
step: 160, loss: 1.510468, accuracy: 0.968750
step: 170, loss: 1.516033, accuracy: 0.960938
step: 180, loss: 1.510814, accuracy: 0.968750
step: 190, loss: 1.513747, accuracy: 0.960938
step: 200, loss: 1.485455, accuracy: 0.992188


In [ ]:
pred = conv_net(x_test)

print('test accuracy: %f' % accuracy(pred, y_test ))

test accuracy: 0.977800
